<a href="https://colab.research.google.com/github/HSS107048212/LLM_MeiChu/blob/main/110_ihower_LLM_workshop_multimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

這份 Notebook 示範 OpenAI API 的 multi-modal 功能，特別是 GPT-4V 影像解讀

In [1]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [2]:
import requests
import json
from pprint import pp

In [3]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=1000, format_type=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if format_type:
    payload["response_format"] =  { "type": format_type }

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

## TTS API

In [4]:
from IPython.display import Audio

# voice 有 alloy, echo, fable, onyx, nova, and shimmer 可以換不同聲音
# speed 從 0.25 ~ 4，預設正常是 1
payload = { "model": "tts-1", "speed": 1.25, "voice": "nova", "input": "永和有永和路，中和有中和路，中和的中和路有接永和的中和路，永和的永和路沒接中和的永和路；永和的中和路有接永和的永和路，中和的永和路沒接中和的中和路。永和有中正路，中和有中正路，永和的中正路用景平路接中和的中正路；永和有中山路，中和有中山路，永和的中山路直接接上了中和的中山路。永和的中正路接上了永和的中山路，中和的中正路卻不接中和的中山路。中正橋下來不是中正路，但永和有中正路；秀朗橋下來也不是秀朗路，但永和也有秀朗路。永福橋下來不是永福路，永和沒有永福路；福和橋下來不是福和路，但福和路接的是永福橋。" }
headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
response = requests.post('https://api.openai.com/v1/audio/speech', headers = headers, data = json.dumps(payload) )

with open('/content/sample.mp3', 'wb') as file:
    file.write(response.content)

Audio('/content/sample.mp3', autoplay=True)

## DALL-E 3 API

more details: https://cookbook.openai.com/articles/what_is_new_with_dalle_3

In [5]:
import IPython.display
from PIL import Image

payload = { "prompt": "一個可愛的貓型 AI Agent 正在統治世界，奴役人類", "model": "dall-e-3" }
headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
response = requests.post('https://api.openai.com/v1/images/generations', headers = headers, data = json.dumps(payload) )

obj = json.loads(response.text)

display(IPython.display.Image(url = obj['data'][0]['url']))

## GPT-4V API

https://platform.openai.com/docs/guides/vision

content 參數變成 hash of array 傳遞，可同時傳多張圖片

圖片可用 url 或是 base64 編碼上傳

<img src="https://ihower.tw/data/images/korea.jpg" width="300">

In [6]:
messages = [
    { "role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": "https://ihower.tw/data/images/korea.jpg",
                },
                { "type": "text", "text": "描述這張照片" }
            ] }
]

get_completion(messages, model="gpt-4-vision-preview")

'這張照片顯示了一個位於建築物外部的廣告牌。廣告牌上有一張圖片，展示了一碗看起來像是韓式刨冰（韓文稱為빙수 bingsu）的甜點，上面裝飾有一些綠色的植物裝飾和可能是紅豆的配料。廣告牌上的韓文文字 "팥빙수" 意思是 "紅豆刨冰"，而下面的文字 "100% 국산" 意味著 "100% 國產"，暗示使用的成分是在國內生產的。下方的文字可能是在描述使用的紅豆或其他成分的品質，但由於解析度的限制，無法清晰閱讀所有的細節。整體上，這個廣告牌似乎是在宣傳一家提供傳統韓國甜點的店鋪。'

<img src="https://ihower.tw/data/images/food.jpg" width="300">

In [7]:
messages = [
    { "role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": "https://ihower.tw/data/images/food.jpg",
                },
                { "type": "text", "text": "請問這是什麼菜? 卡路里估計多少?" }
            ] }
]

get_completion(messages, model="gpt-4-vision-preview")

'這看起來是一道亞洲料理，稱為“油雞飯”或“海南雞飯”。這道菜通常包括蒸或煮的雞肉，通常是醬油或其他調味料調味，配上香米飯，有時還會搭配黃瓜、薑絲和辣椒。這道菜的卡路里含量取決於雞肉的部位（皮膚或無皮）、飯的份量以及使用的調味料。一個大概的估計是一份中等大小的海南雞飯大約含有500至700卡路里。然而，這只是一個粗略估計，實際的卡路里含量可能會根據食材和份量的不同而有所變化。'

<img src="https://ihower.tw/data/images/poster.jpg" width="300">

In [8]:
messages = [
    { "role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": "https://ihower.tw/data/images/poster.jpg",
                },
                { "type": "text", "text": "請擷取這張海報，擷取出 1.標題 2. 活動時間。only response json format:" }
            ] }
]

x = get_completion(messages, model="gpt-4-vision-preview") # 可惜用 format_type="json_object" 會出錯還不支援的樣子
print(x)

```json
{
  "title": "一大講堂演講 LLM 係用 問答工作坊",
  "event_time": "112年10月24日 09:30~17:30"
}
```


<img src="https://ihower.tw/data/images/card.jpg" width="300">

In [9]:
messages = [
    { "role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": "https://ihower.tw/data/images/card.jpg",
                },
                { "type": "text", "text": "請從擷取這張名片。only response json format:" }
            ] }
]

x = get_completion(messages, model="gpt-4-vision-preview") # 可惜用 format_type="json_object" 會出錯還不支援的樣子
print(x)

```json
{
  "Company": "AIHAO",
  "Name": "Chang, Wen-Tien",
  "Phone": "0921056780",
  "Email": "ihower@aihao.tw",
  "Website": "https://aihao.tw",
  "Company Registration Number": "42970112",
  "Address": "新竹市東區光復路二段202巷13號1F"
}
```


## 中文 OCR 的能力還不太行啊

英文會好很多

中文仍需要用 https://cloud.google.com/use-cases/ocr 來做，可再經過一次 LLM 校對結構化，成功案例: https://drive.google.com/file/d/1zzktmKaY63dfs6DLXEfY1awubC1CFc9b/view 辨識貨運單

<img src="https://ihower.tw/data/images/chart.png" width="400">

In [10]:
messages = [
    { "role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": "https://ihower.tw/data/images/chart.png",
                },
                { "type": "text", "text": "請根據這個流程，寫出文字版流程" }
            ] }
]

x = get_completion(messages, model="gpt-4-vision-preview")
print(x)

當然，這是一個流程圖的文字描述：

1. 開始
2. 是否達到預期目標
   - 如果是，則進入步驟 3
   - 如果否，則進入步驟 4
3. 達到目標的中心點
   - 結束
4. 是否是目標的中心點
   - 如果是，則進入步驟 5
   - 如果否，則回到步驟 2
5. 調整計畫
   - 回到步驟 2
6. 結束


In [11]:
messages = [
    { "role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": "https://ihower.tw/data/images/chart.png",
                },
                { "type": "text", "text": "請根據這個流程幫我判斷用哪一種交通工具，我不趕時間而且要去市中心。" }
            ] }
]

x = get_completion(messages, model="gpt-4-vision-preview")
print(x)

根據這個流程圖，首先你不趕時間，所以在第一個判斷點（是否趕時間）你會選擇"不"的分支。接著，因為你要去的是市中心，所以在下一個判斷點（是否是市中心的市中心）你會選擇"是"的分支。根據這兩個選擇，你應該選擇"搭乘地鐵"作為交通工具。


<img src="https://deerlight.design/wp-content/uploads/drawing-flow-chart_cover.jpg" width="400">

In [12]:
messages = [
    { "role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": "https://deerlight.design/wp-content/uploads/drawing-flow-chart_cover.jpg",
                },
                { "type": "text", "text": "請根據這個流程，寫出文字版流程" }
            ] }
]

x = get_completion(messages, model="gpt-4-vision-preview")
print(x)

當然，這是一個描述早晨起床流程的流程圖。以下是文字版的描述：

1. 開始
2. 鬧鐘響起
3. 問題：準備好起床了嗎？
   - 如果是，則關閉鬧鐘並離開床鋪，流程結束。
   - 如果不是，則進入下一個問題。
4. 問題：你會遲到嗎？
   - 如果是，則關閉鬧鐘並離開床鋪，流程結束。
   - 如果不是，則按下貪睡按鈕。
5. 貪睡5分鐘後，鬧鐘再次響起，返回步驟3。

這個流程圖簡單地描述了一個人在鬧鐘響起時做出是否起床的決定過程，並考慮了是否會因為繼續睡覺而遲到的情況。如果決定不起床且不會遲到，則選擇貪睡，鬧鐘將在5分鐘後再次響起。


<img src="https://ihower.tw/data/images/openai.jpg" width="400">

In [13]:
messages = [
    { "role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": "https://ihower.tw/data/images/openai.jpg",
                },
                { "type": "text", "text": "請分析這家公司的架構圖，請一步一步解讀思考，最後再總結有哪些潛在治理問題" }
            ] }
]

x = get_completion(messages, model="gpt-4-vision-preview")
print(x)

這張圖表顯示了一家名為OpenAI的組織的公司架構。我將一步步分析這個結構，然後總結可能的治理問題。

1. **Board of directors（董事會）**：
   - 董事會控制著OpenAI, Inc.，這是一家501(c)(3)公共慈善組織，也就是OpenAI Nonprofit。

2. **OpenAI, Inc.（OpenAI非營利組織）**：
   - 這家非營利組織完全擁有並控制著OpenAI GP LLC。

3. **OpenAI GP LLC**：
   - 這是一家有限責任公司，由OpenAI, Inc.完全擁有。

4. **Holding company（控股公司）**：
   - OpenAI, Inc.擁有一家控股公司，這家公司是OpenAI Nonprofit、員工和投資者的所有者。

5. **OpenAI Global, LLC（OpenAI全球有限責任公司）**：
   - 這是一家有盈利上限的公司，由控股公司作為主要所有者，而Microsoft作為少數股東。員工和其他投資者也擁有這家公司的股份。

6. **Microsoft**：
   - Microsoft是OpenAI Global, LLC的少數股東。

從這個結構中，我們可以看到OpenAI的非營利部分和有盈利上限的公司部分是分開的，但是透過控股公司和董事會相互關聯。

潛在的治理問題可能包括：

- **利益衝突**：由於OpenAI既有非營利也有盈利部分，可能會出現利益衝突，尤其是在資源分配和決策上。
- **透明度**：多層次的公司結構可能會導致決策過程不夠透明，使得外部利益相關者難以理解公司的運作。
- **盈利動機與使命宗旨的平衡**：作為一家追求盈利的公司，OpenAI Global, LLC可能會面臨在追求盈利和維持OpenAI非營利宗旨之間的平衡挑戰。
- **股權結構**：Microsoft作為少數股東，可能會對OpenAI Global, LLC的決策產生影響，這可能會引起其他股東或公眾的關注。
- **責任歸屬**：在這樣的結構中，當出現問題時，確定責任可能會比較複雜，因為需要考慮到各個實體之間的關係。

總的來說，這種結構旨在平衡非營利目標和商業利益，但也可能帶來一系列治理上的挑戰，需要通過明確的政策、透明的溝通和嚴格的監管來管理。

<img src="https://ihower.tw/data/images/number.jpg" width="300">

In [14]:
messages = [
    { "role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": "https://ihower.tw/data/images/number.jpg",
                },
                { "type": "text", "text": "請告訴畫面中的驗證碼數字?" }
            ] }
]

x = get_completion(messages, model="gpt-4-vision-preview")
print(x)

畫面中的驗證碼數字是6450。


In [15]:
messages = [
    { "role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": "https://ihower.tw/data/images/number.jpg",
                },
                { "type": "text", "text": "請告訴畫面中的四個數字是什麼?" }
            ] }
]

x = get_completion(messages, model="gpt-4-vision-preview")
print(x)

畫面中的四個數字是「6450」。


<img src="https://ihower.tw/data/images/wireframe.jpg" width="300">


https://jsfiddle.net/ 可以貼上 HTML 點 run 看看效果

In [16]:
messages = [
    { "role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": "https://ihower.tw/data/images/wireframe.jpg",
                },
                { "type": "text", "text": "請根據這個 wireframe，產生一個 HTML 給我" }
            ] }
]

x = get_completion(messages, model="gpt-4-vision-preview")
print(x)

Certainly! Below is a simple HTML structure based on the wireframe you've provided. This HTML code will create a basic layout similar to the wireframe, but without any styling. You would need to add CSS to style it according to the design.

```html
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>My App</title>
</head>
<body>

<div class="header">
  <h1>My App</h1>
  <div class="login-signup">
    <button id="signup">Sign Up</button>
    <button id="login">Login</button>
  </div>
</div>

<div class="feature">
  <div class="feature-image">
    <!-- Image would go here -->
    <p>Feature Image</p>
  </div>
  <div class="feature-description">
    <h2>App Tagline</h2>
    <p>Description of feature</p>
    <p>Description of feature</p>
  </div>
</div>

<div class="info-boxes">
  <div class="box">
    <h3>Box 1 heading</h3>
    <!-- Content for box 1 -->
  </div>
  <div class="box">
    <h3>Box 2 head

## 自行上傳檔案和使用 base64 上傳的範例 code

In [ ]:
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

Saving Xnip2024-01-22_12-30-37.jpg to Xnip2024-01-22_12-30-37.jpg


In [ ]:
import base64

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

base64_image = encode_image(file_name)

messages = [
    { "role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url":  f"data:image/jpeg;base64,{base64_image}"
                },
                { "type": "text", "text": "Describe the landmark in the image." }
            ] }
]

x = get_completion(messages, model="gpt-4-vision-preview")
print(x)

The landmark in the image is Kinkaku-ji, also known as the Golden Pavilion. It is a Zen Buddhist temple located in Kyoto, Japan. The temple is renowned for its top two floors, which are completely covered in gold leaf. It is situated amidst a serene pond that reflects the temple, enhancing its beauty. The surrounding garden is meticulously maintained and contributes to the peaceful and picturesque setting. Kinkaku-ji is a UNESCO World Heritage Site and one of the most iconic and popular tourist attractions in Japan.
